<a href="https://colab.research.google.com/github/JoshuaHaga/covid_tracking_project/blob/master/Harrisonburg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px

In [4]:
###Must run for visualizations
!pip install plotly==4.9.0

     |████████████████████████████████| 12.9MB 317kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [5]:
#Dependencies for chloropleth
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [6]:
!pip install chart_studio

     |████████████████████████████████| 71kB 2.0MB/s 


In [7]:
import plotly
import chart_studio
import chart_studio.plotly as py
username = 'josh.haga' # your username
api_key = '6bgoxTaL6JLm9a9DquYX' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [8]:
###Datasets
va_locality_cases = pd.read_csv('https://raw.githubusercontent.com/JoshuaHaga/covid_tracking_project/master/VDH-COVID-19-PublicUseDataset-Cases.csv')
va_cases_zip      = pd.read_csv('https://raw.githubusercontent.com/JoshuaHaga/covid_tracking_project/master/VDH-COVID-19-PublicUseDataset-ZIPCode.csv')

In [9]:
va_cases_zip.head()

,Report Date,ZIP Code,Number of Cases,Number of Testing Encounters,Number of PCR Testing Encounters
0,05/15/2020,20105,61,507.0,NaN
1,05/15/2020,20106,25,92.0,NaN
2,05/15/2020,20109,542,1411.0,NaN
3,05/15/2020,20110,609,1588.0,NaN
4,05/15/2020,20111,426,1160.0,NaN


In [10]:
###Filter for just harrisonburg
harrison_zip = [22801,22802,22803,22807]
harrison_cases = va_cases_zip[va_cases_zip['ZIP Code'].isin(['22801','22802','22803','22807'])]
harrison_cases

,Report Date,ZIP Code,Number of Cases,Number of Testing Encounters,Number of PCR Testing Encounters
217,05/15/2020,22801,438,945.0,NaN
218,05/15/2020,22802,363,877.0,NaN
219,05/15/2020,22807,Suppressed*,2.0,NaN
1097,05/16/2020,22801,443,981.0,NaN
1098,05/16/2020,22802,369,900.0,NaN
...,...,...,...,...,...
95721,09/02/2020,22802,656,NaN,4780
95722,09/02/2020,22807,20,NaN,68
96618,09/03/2020,22801,1218,NaN,6954
96619,09/03/2020,22802,660,NaN,4806


###va_locality_cases

In [11]:
va_locality_cases.head()

,Report Date,FIPS,Locality,VDH Health District,Total Cases,Hospitalizations,Deaths
0,03/17/2020,51001,Accomack,Eastern Shore,0,0,0
1,03/17/2020,51003,Albemarle,Thomas Jefferson,0,0,0
2,03/17/2020,51005,Alleghany,Alleghany,0,0,0
3,03/17/2020,51007,Amelia,Piedmont,0,0,0
4,03/17/2020,51009,Amherst,Central Virginia,0,0,0


In [12]:
#va_comb = pd.merge(va_locality_cases,va_cases_zip, on=)

In [13]:
va_locality_cases['Report Date'].describe()

count          22743
unique           171
top       09/03/2020
freq             133
Name: Report Date, dtype: object

In [14]:
current_cases = va_locality_cases[va_locality_cases['Report Date'] == va_locality_cases['Report Date'].max()]

In [16]:
va_county_cases_fig = px.choropleth(va_locality_cases, geojson=counties, locations='FIPS', color='Total Cases',
 color_continuous_scale='portland',
 range_color=(va_locality_cases['Total Cases'].min(), va_locality_cases['Total Cases'].mean()*4),
 scope='usa',
 labels={'color':'Current Covid Cases',
         'Locality':'County',
         }
 )
va_county_cases_fig.update_geos(fitbounds="locations", visible=False)
va_county_cases_fig.update_layout(
    title_text = 'Positive Results by County in VA'
)
#va_county_cases_fig.show()
#va_county_cases_fig.write_html('/content/full_map.html')

TypeError: ignored

In [ ]:
def rolling_seven_avg(df,locality,feature,days):
  df = df[df['Locality']==locality]
  df['new_cases'] = df[feature].diff()
  rolling_avg = df['new_cases'].rolling(window=days).mean()
  df['rolling_avg'] = rolling_avg
  return df
 

In [ ]:
def col_list(dataframe):
  val_counts = dataframe.value_counts()
  val_as_pd  = pd.DataFrame(val_counts).T
  val_list   = val_as_pd.columns.tolist()
  return val_list

In [ ]:
harrison = rolling_seven_avg(va_locality_cases,'Harrisonburg','Total Cases',7)

In [ ]:
#harrison['rolling_avg'].plot(kind='bar',)

In [ ]:
harrisonburg_sevenday_rolling = px.bar(harrison, x='Report Date', y='rolling_avg',
                                       labels={'rolling_avg':'Number of Positive Cases', 
                                      'Report Date':'Date',})
harrisonburg_sevenday_rolling.show()

In [ ]:
py.plot(harrisonburg_sevenday_rolling, filename = 'harrisonburg_sevenday_rolling', auto_open=True)

In [ ]:
augusta = rolling_seven_avg(va_locality_cases,'Augusta','Total Cases',7)

In [ ]:
augusta_sevenday_rolling = px.bar(augusta, x='Report Date', y='rolling_avg',
                                  labels={'rolling_avg':'Number of Positive Cases', 
                                      'Report Date':'Date',
                                      })
#augusta_sevenday_rolling.show()

# Visualizations

###Harrisonburg 7 day Rolling Avg

In [ ]:
harrisonburg_sevenday_rolling.show()

In [ ]:
####Link to embed Harrisonburg 7 day Rolling Avg
py.plot(harrisonburg_sevenday_rolling, filename = 'harrisonburg_sevenday_rolling', auto_open=True)

###VA Cases by county 

In [ ]:
va_county_cases_fig.show()

In [ ]:
py.plot(va_county_cases_fig, filename = 'va_county_cases_fig', auto_open=True)